In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!pip install -q diffusers transformers accelerate safetensors pillow moviepy

In [ ]:
import json
import torch
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont

from diffusers import StableDiffusionPipeline
from moviepy.editor import ImageClip, concatenate_videoclips

In [ ]:
MODEL_ID = "rupeshs/LCM-runwayml-stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    safety_checker=None
).to("cuda")

pipe.enable_xformers_memory_efficient_attention()
pipe.set_progress_bar_config(disable=True)

print("✅ Stable Diffusion loaded")

In [ ]:
from google.colab import files

uploaded = files.upload()

assert "content_job.json" in uploaded, "❌ Upload content_job.json"
print("✅ content_job.json uploaded")


In [ ]:
with open("content_job.json") as f:
    job = json.load(f)

JOB_ID = job["job_id"]

OUTPUT_DIR = Path(f"/content/drive/MyDrive/instagram_ai/{JOB_ID}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("📁 Output directory:", OUTPUT_DIR)


In [ ]:
def generate_image(prompt, output_path, size):
    image = pipe(
        prompt=prompt,
        num_inference_steps=4,
        guidance_scale=1.0
    ).images[0]

    image = image.resize(size)
    image.save(output_path)


In [ ]:
def add_text_overlay(image_path, text):
    img = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(img)

    font = ImageFont.load_default()
    w, h = img.size

    text_w, text_h = draw.textsize(text, font=font)
    x = (w - text_w) // 2
    y = h - text_h - 40

    draw.rectangle(
        [x - 10, y - 10, x + text_w + 10, y + text_h + 10],
        fill=(0, 0, 0)
    )
    draw.text((x, y), text, fill="white", font=font)

    img.save(image_path)


In [ ]:
for post in job["posts"]:
    post_dir = OUTPUT_DIR / post["post_id"]
    post_dir.mkdir(exist_ok=True)

    print(f"🚀 Processing {post['post_id']} ({post['post_type']})")

    # ---------- REELS ----------
    if post["post_type"] == "reel":
        clips = []

        for scene in post["reel"]["scenes"]:
            img_path = post_dir / f"scene_{scene['scene_id']}.png"

            generate_image(
                scene["image_prompt"],
                img_path,
                size=(512, 912)  # 9:16
            )

            add_text_overlay(img_path, scene["overlay_text"])

            clip = ImageClip(str(img_path)).set_duration(scene["duration"])
            clips.append(clip)

        final_video = concatenate_videoclips(clips, method="compose")

        video_path = post_dir / "reel.mp4"
        final_video.write_videofile(
            str(video_path),
            fps=24,
            codec="libx264",
            audio=False
        )

    # ---------- CAROUSEL ----------
    else:
        for slide in post["carousel"]["slides"]:
            img_path = post_dir / f"slide_{slide['slide_id']}.png"

            generate_image(
                slide["image_prompt"],
                img_path,
                size=(512, 640)  # 4:5
            )


In [ ]:
print("✅ ALL CONTENT GENERATED")
print("📂 Google Drive → instagram_ai →", JOB_ID)